In [1]:
""" Ingest PCRGLOBWB timeseries data on Google Earth Engine
-------------------------------------------------------------------------------
This notebook will upload the geotiff files from the Google Cloud Storage to
the WRI/aqueduct earthengine bucket. An errorlog will be stored on Amazon S3.

Requirements:
    Authorize earthengine by running in your terminal: earthengine 
                                                       authenticate

    you need to have access to the WRI-Aquaduct (yep a Google employee made a
    typo) bucket to ingest the data. Rutger can grant access to write to this 
    folder. 

    Have access to the Google Cloud Storage Bucker

Make sure to set the project to Aqueduct30 by running
`gcloud config set project aqueduct30`

Code follows the Google for Python Styleguide. Exception are the scripts that 
use earth engine since this is camelCase instead of underscore.

Author: Rutger Hofste
Date: 20170802
Kernel: python27
Docker: rutgerhofste/gisdocker:ubuntu16.04

Args:    
    TESTING (Boolean) : Toggle Testing Mode.
    OVERWRITE (Boolean) : Overwrite old folder !CAUTION!
    
    SCRIPT_NAME (string) : Script name.    
    PREVIOUS_SCRIPT_NAME (string) : Previous script name. 
    INPUT_VERSION (integer) : Input version.
    EE_OUTPUT_VERSION (integer) : Output version for earthengine. 
    OUTPUT_VERSION (integer) : Output version.     
    OUTPUT_FILE_NAME (string) : File Name for a csv file containing the failed tasks. 
    
    SEPARATOR (regex) : Regular expression of separators used in geotiff
      filenames.     
    SCHEMA (list) : A list of strings containing the schema. See 
      aqueduct3.split_key() for more info.
    EXTRA_PROPERTIES (Dictionary) : Extra properties to add to assets. nodata_value,
      script used are common properties.

Returns:


"""

# Input Parameters
TESTING = 0
OVERWRITE = 1 # !CAUTION!
SCRIPT_NAME = "Y2017M08D02_RH_Ingest_GCS_EE_V02"
PREVIOUS_SCRIPT_NAME = "Y2017M07D31_RH_Convert_NetCDF_Geotiff_V02"

INPUT_VERSION = 2
EE_OUTPUT_VERSION = 9
OUTPUT_VERSION = 9

OUTPUT_FILE_NAME = "df_errorsV01.csv"

SEPARATOR = "_|-"
SCHEMA = ["geographic_range",
     "temporal_range",
     "indicator",
     "temporal_resolution",
     "unit",
     "spatial_resolution",
     "temporal_range_min",
     "temporal_range_max"]

EXTRA_PROPERTIES = {"nodata_value":-9999,
                    "ingested_by" : "RutgerHofste",
                    "script_used": SCRIPT_NAME,
                    "output_version":OUTPUT_VERSION}}

# ETL
gcs_input_path = "gs://aqueduct30_v01/{}/output_V{:02.0f}/".format(PREVIOUS_SCRIPT_NAME,INPUT_VERSION)
ee_output_path = "projects/WRI-Aquaduct/PCRGlobWB20V{:02.0f}".format(EE_OUTPUT_VERSION)
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)
ec2_output_path = "/volumes/data/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)

print("Input gcs: " +  gcs_input_path +
      "\nOutput ee: " + ee_output_path +
      "\nOutput S3: " + s3_output_path +
      "\nOutput ec2: " + ec2_output_path)


Input gcs: gs://aqueduct30_v01/Y2017M07D31_RH_Convert_NetCDF_Geotiff_V02/output_V02/
Output ee: projects/WRI-Aquaduct/PCRGlobWB20V09
Output S3: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Ingest_GCS_EE_V02/output_V09
Output ec2: /volumes/data/Y2017M08D02_RH_Ingest_GCS_EE_V02/output_V09


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M04D17 UTC 12:05


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
# Imports
import subprocess
import datetime
import os
import time
import re
import pandas as pd
from datetime import timedelta
import aqueduct3


def main():
    start_time = time.time()
    !mkdir -p {ec2_output_path}
    
    keys = aqueduct3.get_GCS_keys(gcs_input_path)
    df = aqueduct3.keys_to_df(keys,SEPARATOR,SCHEMA)
    df = df.assign(**EXTRA_PROPERTIES)
    df["exportdescription"] = df["indicator"] + "_" + df["temporal_resolution"]+"Y"+df["year"]+"M"+df["month"]
    df = df.apply(pd.to_numeric, errors='ignore')
    
    # Earth Engine Preparations
    # Create folder
    if OVERWRITE:
        command = "earthengine rm -r {}".format(ee_output_path)
        print(command)
        subprocess.check_output(command,shell=True)

    command = "earthengine create folder {}".format(ee_output_path)
    print(command)
    subprocess.check_output(command,shell=True)
    
    # Create ImageCollections
    parameters = df.parameter.unique()
    for parameter in parameters:
        ic_id = ee_output_path + "/" + parameter
        command, result = aqueduct3.create_imageCollection(ic_id)
        print(command,result)

    if TESTING:
        df = df[1:3] 
       
    df_errors = pd.DataFrame()
    
    for index, row in df.iterrows():
        elapsed_time = time.time() - start_time 
        print(index,"{:02.2f}".format((float(index)/df.shape[0])*100) + "elapsed: ", str(timedelta(seconds=elapsed_time)))

        geotiff_gcs_path = gcs_input_path + row.file_name + "." + row.extension
        output_ee_asset_id = ee_output_path +"/"+ row.parameter + "/" + row.file_name
        properties = row.to_dict()

        df_errors2 = aqueduct3.upload_geotiff_to_EE_imageCollection(geotiff_gcs_path, output_ee_asset_id, properties,index)
        df_errors = df_errors.append(df_errors2)    

    # Storing error dataframe on ec2 and S3
    df_errors.to_csv("{}/{}".format(ec2_output_path,OUTPUT_FILE_NAME))
    !aws s3 cp  {ec2_output_path} {s3_output_path} --recursive
    
    # Retry Failed Tasks Once
    df_retry = df_errors.loc[df_errors['error'] != 0]
    for index, row in df_retry.iterrows():
        response = subprocess.check_output(row.command, shell=True)
    
    return df,df_errors



if __name__ == "__main__":
    df,df_errors = main()

earthengine rm -r projects/WRI-Aquaduct/PCRGlobWB20V09
earthengine create folder projects/WRI-Aquaduct/PCRGlobWB20V09
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWN_month_millionm3_5min_1960_2014 b''
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWN_year_millionm3_5min_1960_2014 b''
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_month_millionm3_5min_1960_2014 b''
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PDomWW_year_millionm3_5min_1960_2014 b''
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_month_millionm3_5min_1960_2014 b''
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWN_year_millionm3_5min_1960_2014 b''
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PIndWW_month_millionm3_5min_1960_2014 

151 1.63elapsed:  0:10:11.823550
152 1.64elapsed:  0:10:14.790312
153 1.65elapsed:  0:10:17.765038
154 1.66elapsed:  0:10:20.726679
155 1.67elapsed:  0:10:23.714217
156 1.68elapsed:  0:10:26.760054
157 1.69elapsed:  0:10:29.716276
158 1.70elapsed:  0:10:32.720433
159 1.71elapsed:  0:10:35.686680
160 1.72elapsed:  0:10:38.679578
161 1.73elapsed:  0:10:41.638911
162 1.74elapsed:  0:10:44.625741
163 1.75elapsed:  0:10:47.668477
164 1.77elapsed:  0:10:50.647356
165 1.78elapsed:  0:10:53.547140
166 1.79elapsed:  0:10:56.522378
167 1.80elapsed:  0:10:59.609974
168 1.81elapsed:  0:11:02.637070
169 1.82elapsed:  0:11:05.585962
170 1.83elapsed:  0:11:08.566226
171 1.84elapsed:  0:11:11.635405
172 1.85elapsed:  0:11:14.633631
173 1.86elapsed:  0:11:17.674287
174 1.87elapsed:  0:11:20.641876
175 1.88elapsed:  0:11:23.630862
176 1.89elapsed:  0:11:26.584303
177 1.91elapsed:  0:11:29.600856
178 1.92elapsed:  0:11:32.560139
179 1.93elapsed:  0:11:35.545442
180 1.94elapsed:  0:11:37.594592
181 1.95el

400 4.31elapsed:  0:22:37.418408
401 4.32elapsed:  0:22:40.443687
402 4.33elapsed:  0:22:43.522055
403 4.34elapsed:  0:22:46.537966
404 4.35elapsed:  0:22:49.497252
405 4.36elapsed:  0:22:52.475825
406 4.37elapsed:  0:22:55.503042
407 4.38elapsed:  0:22:58.021716
408 4.39elapsed:  0:23:01.027172
409 4.40elapsed:  0:23:03.597609
410 4.41elapsed:  0:23:06.615721
411 4.42elapsed:  0:23:09.714151
412 4.43elapsed:  0:23:12.741830
413 4.45elapsed:  0:23:15.787932
414 4.46elapsed:  0:23:18.806468
415 4.47elapsed:  0:23:22.403629
416 4.48elapsed:  0:23:25.473735
417 4.49elapsed:  0:23:28.554247
418 4.50elapsed:  0:23:31.628558
419 4.51elapsed:  0:23:34.705945
420 4.52elapsed:  0:23:37.732014
421 4.53elapsed:  0:23:40.762403
422 4.54elapsed:  0:23:43.823655
423 4.55elapsed:  0:23:46.837984
424 4.56elapsed:  0:23:49.869801
425 4.57elapsed:  0:23:52.858076
426 4.59elapsed:  0:23:55.897980
427 4.60elapsed:  0:23:58.929534
428 4.61elapsed:  0:24:01.959420
429 4.62elapsed:  0:24:05.018143
430 4.63el

649 6.99elapsed:  0:35:00.784232
650 7.00elapsed:  0:35:03.475062
651 7.01elapsed:  0:35:05.717288
652 7.02elapsed:  0:35:08.870193
653 7.03elapsed:  0:35:12.095808
654 7.04elapsed:  0:35:14.946220
655 7.05elapsed:  0:35:18.170327
656 7.06elapsed:  0:35:20.493841
657 7.07elapsed:  0:35:23.689236
658 7.08elapsed:  0:35:26.945118
659 7.09elapsed:  0:35:29.667682
660 7.10elapsed:  0:35:32.375513
661 7.12elapsed:  0:35:35.562485
662 7.13elapsed:  0:35:38.573915
663 7.14elapsed:  0:35:41.333505
664 7.15elapsed:  0:35:44.067005
665 7.16elapsed:  0:35:46.827992
666 7.17elapsed:  0:35:49.920641
667 7.18elapsed:  0:35:52.930282
668 7.19elapsed:  0:35:55.969586
669 7.20elapsed:  0:35:58.996554
670 7.21elapsed:  0:36:02.046353
671 7.22elapsed:  0:36:05.095048
672 7.23elapsed:  0:36:08.158027
673 7.24elapsed:  0:36:11.190384
674 7.26elapsed:  0:36:14.219989
675 7.27elapsed:  0:36:16.804861
676 7.28elapsed:  0:36:19.467293
677 7.29elapsed:  0:36:22.469265
678 7.30elapsed:  0:36:25.524932
679 7.31el

898 9.67elapsed:  0:47:18.607014
899 9.68elapsed:  0:47:21.626456
900 9.69elapsed:  0:47:24.691189
901 9.70elapsed:  0:47:27.700386
902 9.71elapsed:  0:47:30.717108
903 9.72elapsed:  0:47:33.710338
904 9.73elapsed:  0:47:36.707772
905 9.74elapsed:  0:47:39.733753
906 9.75elapsed:  0:47:42.729410
907 9.76elapsed:  0:47:45.789091
908 9.77elapsed:  0:47:48.858268
909 9.78elapsed:  0:47:51.938946
910 9.80elapsed:  0:47:54.967789
911 9.81elapsed:  0:47:57.992929
912 9.82elapsed:  0:48:01.010272
913 9.83elapsed:  0:48:04.016835
914 9.84elapsed:  0:48:07.012321
915 9.85elapsed:  0:48:10.050590
916 9.86elapsed:  0:48:13.037291
917 9.87elapsed:  0:48:16.053695
918 9.88elapsed:  0:48:19.077108
919 9.89elapsed:  0:48:22.140015
920 9.90elapsed:  0:48:24.712170
921 9.91elapsed:  0:48:27.751198
922 9.92elapsed:  0:48:30.807341
923 9.94elapsed:  0:48:33.853813
924 9.95elapsed:  0:48:36.866518
925 9.96elapsed:  0:48:39.914824
926 9.97elapsed:  0:48:42.906360
927 9.98elapsed:  0:48:45.912004
928 9.99el

...

8186 88.12elapsed:  6:54:10.547670
8187 88.13elapsed:  6:54:13.211345
8188 88.14elapsed:  6:54:15.818667
8189 88.15elapsed:  6:54:18.820381
8190 88.16elapsed:  6:54:21.856314
8191 88.17elapsed:  6:54:24.859132
8192 88.18elapsed:  6:54:27.976714
8193 88.19elapsed:  6:54:31.017978
8194 88.20elapsed:  6:54:34.066947
8195 88.21elapsed:  6:54:37.024814
8196 88.22elapsed:  6:54:40.056767
8197 88.23elapsed:  6:54:43.073695
8198 88.25elapsed:  6:54:46.274576
8199 88.26elapsed:  6:54:49.310740
8200 88.27elapsed:  6:54:52.386932
8201 88.28elapsed:  6:54:55.413060
8202 88.29elapsed:  6:54:58.465002
8203 88.30elapsed:  6:55:01.511746
8204 88.31elapsed:  6:55:04.564849
8205 88.32elapsed:  6:55:07.108325
8206 88.33elapsed:  6:55:10.146574
8207 88.34elapsed:  6:55:13.164959
8208 88.35elapsed:  6:55:16.169414
8209 88.36elapsed:  6:55:19.155855
8210 88.37elapsed:  6:55:22.173887
8211 88.39elapsed:  6:55:25.211195
8212 88.40elapsed:  6:55:28.259593
8213 88.41elapsed:  6:55:31.245542
8214 88.42elapsed:  

8421 90.65elapsed:  7:05:52.199862
8422 90.66elapsed:  7:05:55.266019
8423 90.67elapsed:  7:05:57.802493
8424 90.68elapsed:  7:06:00.773066
8425 90.69elapsed:  7:06:03.772478
8426 90.70elapsed:  7:06:06.706673
8427 90.71elapsed:  7:06:09.773037
8428 90.72elapsed:  7:06:12.836845
8429 90.73elapsed:  7:06:16.133514
8430 90.74elapsed:  7:06:19.161741
8431 90.75elapsed:  7:06:21.885779
8432 90.76elapsed:  7:06:24.922352
8433 90.78elapsed:  7:06:27.959400
8434 90.79elapsed:  7:06:31.035456
8435 90.80elapsed:  7:06:34.031112
8436 90.81elapsed:  7:06:37.085746
8437 90.82elapsed:  7:06:40.128413
8438 90.83elapsed:  7:06:43.159156
8439 90.84elapsed:  7:06:46.197916
8440 90.85elapsed:  7:06:49.239030
8441 90.86elapsed:  7:06:52.268852
8442 90.87elapsed:  7:06:55.292774
8443 90.88elapsed:  7:06:58.349384
8444 90.89elapsed:  7:07:00.944860
8445 90.90elapsed:  7:07:03.997638
8446 90.91elapsed:  7:07:06.546190
8447 90.93elapsed:  7:07:09.588477
8448 90.94elapsed:  7:07:12.597264
8449 90.95elapsed:  

8656 93.18elapsed:  7:17:26.001966
8657 93.19elapsed:  7:17:28.563012
8658 93.20elapsed:  7:17:31.671409
8659 93.21elapsed:  7:17:34.704017
8660 93.22elapsed:  7:17:37.699683
8661 93.23elapsed:  7:17:40.722106
8662 93.24elapsed:  7:17:43.276412
8663 93.25elapsed:  7:17:46.289235
8664 93.26elapsed:  7:17:49.323859
8665 93.27elapsed:  7:17:52.455400
8666 93.28elapsed:  7:17:55.469614
8667 93.29elapsed:  7:17:58.638626
8668 93.30elapsed:  7:18:01.696831
8669 93.32elapsed:  7:18:04.753574
8670 93.33elapsed:  7:18:07.861193
8671 93.34elapsed:  7:18:11.015843
8672 93.35elapsed:  7:18:14.049220
8673 93.36elapsed:  7:18:17.111499
8674 93.37elapsed:  7:18:20.124502
8675 93.38elapsed:  7:18:23.194939
8676 93.39elapsed:  7:18:26.263344
8677 93.40elapsed:  7:18:29.421748
8678 93.41elapsed:  7:18:32.479533
8679 93.42elapsed:  7:18:35.588234
8680 93.43elapsed:  7:18:38.673139
8681 93.44elapsed:  7:18:41.822037
8682 93.46elapsed:  7:18:44.897460
8683 93.47elapsed:  7:18:47.511480
8684 93.48elapsed:  

8891 95.71elapsed:  7:29:06.764169
8892 95.72elapsed:  7:29:09.820099
8893 95.73elapsed:  7:29:12.922961
8894 95.74elapsed:  7:29:15.960181
8895 95.75elapsed:  7:29:18.990863
8896 95.76elapsed:  7:29:21.593134
8897 95.77elapsed:  7:29:24.786781
8898 95.78elapsed:  7:29:28.012421
8899 95.79elapsed:  7:29:31.239474
8900 95.80elapsed:  7:29:34.465717
8901 95.81elapsed:  7:29:37.713984
8902 95.82elapsed:  7:29:40.848157
8903 95.83elapsed:  7:29:44.098638
8904 95.84elapsed:  7:29:47.315730
8905 95.86elapsed:  7:29:50.492935
8906 95.87elapsed:  7:29:53.748769
8907 95.88elapsed:  7:29:56.916198
8908 95.89elapsed:  7:30:00.112606
8909 95.90elapsed:  7:30:03.403804
8910 95.91elapsed:  7:30:06.558681
8911 95.92elapsed:  7:30:09.809283
8912 95.93elapsed:  7:30:13.188112
8913 95.94elapsed:  7:30:16.456657
8914 95.95elapsed:  7:30:19.645095
8915 95.96elapsed:  7:30:22.795776
8916 95.97elapsed:  7:30:25.868564
8917 95.98elapsed:  7:30:28.519037
8918 96.00elapsed:  7:30:31.517900
8919 96.01elapsed:  

9126 98.23elapsed:  7:40:55.551950
9127 98.25elapsed:  7:40:58.641432
9128 98.26elapsed:  7:41:01.771865
9129 98.27elapsed:  7:41:04.900469
9130 98.28elapsed:  7:41:07.968979
9131 98.29elapsed:  7:41:11.028275
9132 98.30elapsed:  7:41:14.119225
9133 98.31elapsed:  7:41:17.204756
9134 98.32elapsed:  7:41:19.354368
9135 98.33elapsed:  7:41:22.409374
9136 98.34elapsed:  7:41:25.515657
9137 98.35elapsed:  7:41:28.786014
9138 98.36elapsed:  7:41:31.845373
9139 98.37elapsed:  7:41:34.837091
9140 98.39elapsed:  7:41:37.834918
9141 98.40elapsed:  7:41:40.921921
9142 98.41elapsed:  7:41:43.883429
9143 98.42elapsed:  7:41:46.917686
9144 98.43elapsed:  7:41:49.977922
9145 98.44elapsed:  7:41:53.049937
9146 98.45elapsed:  7:41:56.072294
9147 98.46elapsed:  7:41:59.038009
9148 98.47elapsed:  7:42:01.538008
9149 98.48elapsed:  7:42:04.553888
9150 98.49elapsed:  7:42:07.530510
9151 98.50elapsed:  7:42:10.000562
9152 98.51elapsed:  7:42:13.000969
9153 98.53elapsed:  7:42:15.547449
9154 98.54elapsed:  

In [4]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

7:49:15.632514


Previous Runs:  
7:49:15.632514